In [1]:
%%time

from core.TrajectoryDF import NumPandasTraj as NumTrajDF
from features.temporal_features import TemporalFeatures as temporal
import pandas as pd

gulls = pd.read_csv('./data/gulls.csv')

gulls = NumTrajDF(gulls,
                 latitude='location-lat',
                 longitude='location-long',
                 datetime='timestamp',
                 traj_id='tag-local-identifier',
                 rest_of_columns=[])
gulls.head()

CPU times: user 965 ms, sys: 659 ms, total: 1.62 s
Wall time: 628 ms


,,event-id,visible,lon,lat,sensor-type,individual-taxon-canonical-name,individual-local-identifier,study-name
DateTime,traj_id,,,,,,,,
2009-05-27 14:00:00,91732,1082620685,True,24.58617,61.24783,gps,Larus fuscus,91732A,Navigation experiments in lesser black-backed ...
2009-05-27 20:00:00,91732,1082620686,True,24.58217,61.23267,gps,Larus fuscus,91732A,Navigation experiments in lesser black-backed ...
2009-05-28 05:00:00,91732,1082620687,True,24.53133,61.18833,gps,Larus fuscus,91732A,Navigation experiments in lesser black-backed ...
2009-05-28 08:00:00,91732,1082620688,True,24.58200,61.23283,gps,Larus fuscus,91732A,Navigation experiments in lesser black-backed ...
2009-05-28 14:00:00,91732,1082620689,True,24.58250,61.23267,gps,Larus fuscus,91732A,Navigation experiments in lesser black-backed ...


In [2]:
%%time

date = temporal.create_date_column(gulls, inplace=True)
date.head()

CPU times: user 250 ms, sys: 111 ms, total: 361 ms
Wall time: 548 ms


,,event-id,visible,lon,lat,sensor-type,individual-taxon-canonical-name,individual-local-identifier,study-name,Date
DateTime,traj_id,,,,,,,,,
2009-05-27 14:00:00,91732,1082620685,True,24.58617,61.24783,gps,Larus fuscus,91732A,Navigation experiments in lesser black-backed ...,2009-05-27
2009-05-27 20:00:00,91732,1082620686,True,24.58217,61.23267,gps,Larus fuscus,91732A,Navigation experiments in lesser black-backed ...,2009-05-27
2009-05-28 05:00:00,91732,1082620687,True,24.53133,61.18833,gps,Larus fuscus,91732A,Navigation experiments in lesser black-backed ...,2009-05-28
2009-05-28 08:00:00,91732,1082620688,True,24.58200,61.23283,gps,Larus fuscus,91732A,Navigation experiments in lesser black-backed ...,2009-05-28
2009-05-28 14:00:00,91732,1082620689,True,24.58250,61.23267,gps,Larus fuscus,91732A,Navigation experiments in lesser black-backed ...,2009-05-28


In [3]:
%%time

pdf = pd.read_csv('./data/geolife_sample.csv')
dataframe = NumTrajDF(pdf,'lat','lon','datetime','id')

CPU times: user 259 ms, sys: 9.39 ms, total: 268 ms
Wall time: 266 ms


In [4]:
%%time

dataframe = temporal.create_date_column(dataframe, inplace=True)

CPU times: user 218 ms, sys: 93.6 ms, total: 311 ms
Wall time: 548 ms


In [5]:
%%time

dataframe = temporal.create_day_of_week_column(dataframe, inplace=True)

CPU times: user 318 ms, sys: 137 ms, total: 454 ms
Wall time: 899 ms


In [6]:
%%time

dataframe = temporal.create_time_column(dataframe, inplace=True)

CPU times: user 466 ms, sys: 177 ms, total: 643 ms
Wall time: 786 ms


In [9]:
%%time

dataframe.to_csv('example.csv')

CPU times: user 4.77 s, sys: 61.7 ms, total: 4.83 s
Wall time: 4.84 s
